# Einlesen der log Dateien aus Bordrechnern, extrahieren der Umsteigehinweise

In [ ]:
import pandas as pd
import numpy as np
import duckdb
import os
import zipfile

In [ ]:
folder_path = 'log'
file_path = 'NetPeerManager.2025-11-13.1.log' #Original IVU Log Datei

## reduzieren des Logs 
- auf Zeilen mit Umsteigehinweisen (Modul 132)
- entfernen nicht relevanter Mandaten EDZ und LSE
- entfernen der Fahrerkennung

In [99]:
def reduce_umsteigehinweise(folder_path, file_path, liste_ausschluss):
    file_path_new = file_path + '.reduced' #Reduzierte Datei mit Umsteigehinweisen Modul 132, ohne LSA/EDZ ohne Fahrernummer
    
    #Löschen wegen append-Mode
    if os.path.exists(os.path.join(folder_path, file_path_new)):
        os.remove(os.path.join(folder_path, file_path_new))

    with open(os.path.join(folder_path, file_path), encoding="Latin1") as f:
        for line in f:
            s =line.split('#')
            if len(s) > 8: #kurze Einträge ohne Umsteigehinweise für die weitere Prüfung nicht berücksichtigen
                if s[1] == '132' and s[3][0:3] not in liste_ausschluss: #Auswahl der Umsteigehinweise Modul 132
                    with open(os.path.join(folder_path, file_path_new), 'a', encoding="Latin1") as f_new:
                        s[3] = s[3][0:3] # nur die ersten 3 Zeichen der Kennung Mandant / Fahrernummer behalten
                        f_new.write('#'.join(s)) # Schreiben der ausgewählten Zeilen in die neue Datei

    #Ausgabee als ZIP Datei
    file_to_zip = os.path.join(folder_path, file_path_new)
    zip_filename = file_to_zip + '.zip'
    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        zipf.write(file_to_zip, arcname=os.path.basename(file_to_zip))
    os.remove(file_to_zip) #Löschen der Datei
    
    return zip_filename, file_to_zip

In [ ]:
zip_filename = reduce_umsteigehinweise(folder_path=folder_path, 
                                       file_path=file_path, 
                                       liste_ausschluss=['LSE', 'EDZ'])

## Auslesen Umsteigehinweise und Parsen

In [ ]:
ln = 0 #laufende Nummer Zeilen
arr_concat = [] #initialisieren der Liste für DataFrames

with zipfile.ZipFile(zip_filename[0], 'r') as zipf:
    with zipf.open(os.path.basename(zip_filename[1])) as f:
        for raw_line in f:
            line = raw_line.decode('latin1').strip()  # decode and strip
            ln += 1 #Zeilennummer
            ts = line[0:29]  #Timestamp extrahieren
            s = line.split('#') #Aufspalten der Zeile in Liste 
            fzg = s[3] #Fahrzeugmandant        

            #print(fzg, ln, len(s), s[0][-1:], len(s)%14, s[1])
            arr = np.array(s[6:-3]) #in NumPy Array umwandeln, nur die Datenfelder ohne Felder zu Fzg etc und ohne die letzten 3 Felder (in der Regel leer)
            arr = arr.reshape(-1,14) #Umformen in Array mit 14 Spalten    
            df = pd.DataFrame(arr) #in DataFrame umwandeln
            df['Fzg'] = fzg #Anfügen der Fahrzeugnummer als neue Spalte      
            df['timestamp'] = ts #Anfügen des Timestamps als neue Spalte
            arr_concat.append(df)

### Erstellen eines Dataframes

In [ ]:
result_df_raw = pd.concat(arr_concat, ignore_index=True)
result_df_raw.columns = ['Timestamp1', 'HaltID', 'Halt', 
                     'Mode', 'Unknown1', 'Unknown2', 'Unknown3', 
                     'LinieIntern', 'Linie', 'HaltId2', 'Halt2', 
                     'Unknown4', 'JsonText', 'Timestamp2', 
                     'Fzg', 'timestamp']

In [ ]:
result_df_raw.shape #Größe der DataFrame anzeigen

In [ ]:
result_df_raw.head()

### Ersetzen der leeren Werte bei HaltID und Halt, Umwandlung der Typen

In [ ]:
result_df = result_df_raw.copy()
result_df.replace({"HaltID": "", "Halt":"0"},None, inplace=True) #leere Strings in None umwandeln

In [ ]:
result_df[['HaltID','Halt']] = result_df[['HaltID','Halt']].ffill(inplace=False) #Forward fill auf Spalten Halt und HaltIDanwenden

In [ ]:
result_df['HaltID'] = result_df['HaltID'].astype('Int64')

In [ ]:
result_df.head()

In [ ]:
result_df['timestamp'] = pd.to_datetime(result_df['timestamp'], errors='raise',format="%Y-%m-%dT%H:%M:%S,%f%z") #in Zeittyp umwandeln

In [ ]:
result_df.head()

### Auswertung der Anzahl nach Linien und Fahrzeugen Dataframe

In [ ]:
# Concatenate all DataFrames in arr_concat into a single DataFrame
result_df['Linie'].value_counts()

In [ ]:
# Concatenate all DataFrames in arr_concat into a single DataFrame
result_df['Fzg'].value_counts()

### Auswertung für Zeitbereich Dataframe

In [ ]:
result_df.timestamp.min(), result_df.timestamp.max()

In [ ]:
zeit_fil = '2025-11-13 05:45:00+0100'
df_zeit = result_df[result_df['timestamp'] >= zeit_fil]
df_zeit

### Ausgabe als Parquet

- komprinierte Ausgabe mit Feldformaten

In [ ]:
file_path_parquet = file_path + '.parquet'

In [ ]:
result_df.to_parquet(os.path.join('out', file_path_parquet), index=False)

## Auswertung in DuckDB

- später als getrenntes Modul

In [ ]:
con = duckdb.connect(database=':memory:')
#einlesen aller parquet aus Ordner ggf. Einschränkung auf Datum/Datumsbereich im Dateinamen
con.execute("CREATE TABLE umstiege AS SELECT * FROM read_parquet('out/*2025-11-13*.parquet');")
con.sql("select count(*) from umstiege")

### Übermittelte Linien je Fzg 

- Ausgabe von Kennwerten je Fahrzeug
- Ermitteln der externen Anschlüsse

In [ ]:
#weitere Linien im Regionalbusbereich, die nicht dreistellige Nummern haben
#Problematische Linien: 63, 63S, 20,21 Linien können sowohl RBL-intern wie auch von der BSAG kommen
weitere_regio = "'S35','3860'"

In [ ]:
con.sql(f"""
    SELECT distinct linie, case when (not REGEXP_MATCHES(linie, '^[0-9]{{3}}$')) and (linie not in ({weitere_regio})) then linie end as linie_ohne_regio
        from umstiege
        -- Beschränkung auf bestimmte Fahrzeuge / Ausschluss LSE Fahrzeuge
        -- wenn nicht bereits beim Export ausgeschlossen
        where fzg not like '%LSE%' 
        order by linie_ohne_regio, linie""").df()

### Anschlüsse je Fahrzeug

In [ ]:
con.sql(f"""
        select distinct fzg, min(timestamp) as min_ts, max(timestamp) as max_ts, count(*) as anzahl,
        count(distinct linie) as linienanzahl,        
        string_agg(distinct linie, ', ' ORDER BY linie) as linien,
        string_agg(distinct linie_ohne_regio, ', ' ORDER BY linie_ohne_regio) as linien_ohne_regio
        
        from 
            -- Tabelle mit zusätzlicher Spalte linie_ohne_regio nicht dreistellige Liniennummern und weitere Regio Linien
            (SELECT *, 
            case when (not REGEXP_MATCHES(linie, '^[0-9]{{3}}$')) and (linie not in ({weitere_regio}))  then linie end as linie_ohne_regio
            from umstiege) 
                
        -- Beschränkung auf bestimmte Fahrzeuge / Ausschluss LSE Fahrzeuge
        -- wenn nicht bereits beim Export ausgeschlossen
        where fzg not like '%LSE%' 
        
        group by fzg
        order by fzg""").df()

### Anschlüsse je Haltepunkt

In [ ]:
con.sql(f"""
        SELECT distinct haltid, halt, count(*) as anzahl_hinweise,
        string_agg(distinct linie, ', ' ORDER BY linie) as linien,
        string_agg(distinct linie_ohne_regio, ', ' ORDER BY linie_ohne_regio) as linien_ohne_regio,
        string_Agg(distinct fzg, ', ' ORDER BY fzg) as fahrzeuge
        from (SELECT *, 
            case when (not REGEXP_MATCHES(linie, '^[0-9]{{3}}$')) and (linie not in ({weitere_regio}))  then linie end as linie_ohne_regio
            from umstiege) 
        -- Beschränkung auf bestimmte Fahrzeuge / Ausschluss LSE Fahrzeuge
        where fzg not like '%LSE%'
        group by haltid, halt
        order by halt, haltid""").df()

### Haltestellen mit externen Anschlüssen

In [ ]:
haltestelle = 'Universität' #Suchbegriff für Haltestelle
con.sql(f"""select * from (
        SELECT distinct haltid, halt, count(*) as anzahl_umstiege,
        string_agg(distinct linie, ', ' ORDER BY linie) as linien,
        string_agg(distinct linie_ohne_regio, ', ' ORDER BY linie_ohne_regio) as linien_ohne_regio,
        string_Agg(distinct fzg, ', ' ORDER BY fzg) as fahrzeuge
        from (SELECT *, 
            case when (not REGEXP_MATCHES(linie, '^[0-9]{{3}}$')) and (linie not in ({weitere_regio}))  then linie end as linie_ohne_regio
            from umstiege) 
        -- Beschränkung auf bestimmte Fahrzeuge / Ausschluss LSE Fahrzeuge
        where fzg not like '%LSE%'
        group by haltid, halt
        order by halt, haltid)
        where (linien_ohne_regio is not Null) and (halt like '%{haltestelle}%')""").df()

In [ ]:
con.close()